In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install flask_restful
!pip install werkzeug
!pip install flask-cors
from flask import Flask ,request,jsonify, send_file
from flask_ngrok import run_with_ngrok
from flask_restful import Resource, Api, reqparse
import subprocess
from flask_cors import CORS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
import shutil
shutil.copytree("/content/drive/MyDrive/thin", "/content/thin")

'/content/thin'

In [ ]:
cd thin

/content/thin


In [ ]:
import torch
# edit the config
device = torch.device('cuda:0')
predict_mode = 'relative' # ['standard', 'relative', 'avd']
find_best_frame = False # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result
if find_best_frame:
  !pip install face_alignment

In [ ]:
from PIL import Image
try:
      import imageio
      import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os

warnings.filterwarnings("ignore")


In [ ]:
pwd

'/content/thin'

In [ ]:
cp "/content/drive/MyDrive/DemoConfig/mgif_taichi_thin_plate.yaml"  /content/thin/config

In [ ]:
cp "/content/drive/MyDrive/DemoConfig/fashion_taichi_thin_plate.yaml"  /content/thin/config

In [ ]:
import thin.demo
inpaintingFashion, kp_detectorFashion, dense_motion_networkFashion, avd_networkFashion = thin.demo.load_checkpoints(config_path = '/content/thin/config/fashion_taichi_thin_plate.yaml', checkpoint_path = '/content/drive/MyDrive/Fashion_Taichi_ThinPlate/00000033-checkpoint.pth.tar', device = device)

In [ ]:
import thin.demo
inpaintingGif, kp_detectorGif, dense_motion_networkGif, avd_networkGif = thin.demo.load_checkpoints(config_path = '/content/thin/config/mgif_taichi_thin_plate.yaml', checkpoint_path = '/content/drive/MyDrive/MGIF_Taichi_ThinPlate/00000032-checkpoint.pth.tar', device = device)

In [ ]:
cd ..

/content


In [ ]:
!git clone https://github.com/wyhsirius/LIA.git

Cloning into 'LIA'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 95 (delta 17), reused 15 (delta 15), pack-reused 73
Unpacking objects: 100% (95/95), 21.49 MiB | 9.25 MiB/s, done.


In [ ]:
!rm /content/LIA/run_demo.py

In [ ]:
cp "/content/drive/MyDrive/run_demo.py"  /content/LIA

In [ ]:
cp "/content/drive/MyDrive/fashion_tarining/taichi/checkpoint/27.pt"  /content/LIA/checkpoints

In [ ]:
cp "/content/drive/MyDrive/mgif_tarining/taichi/checkpoint/30.pt"  /content/LIA/checkpoints

In [ ]:
#ted if fashion, taichi if mgif
import os
old_name = r"/content/LIA/checkpoints/30.pt"
new_name = r"/content/LIA/checkpoints/gif.pt"
os.rename(old_name, new_name)
old_name = r"/content/LIA/checkpoints/27.pt"
new_name = r"/content/LIA/checkpoints/fashion.pt"
os.rename(old_name, new_name)

In [ ]:
!pip install av

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 17.6 MB/s eta 0:00:00


In [ ]:
pwd


'/content'

In [ ]:
server = Flask (__name__)
run_with_ngrok(server)
CORS(server)


@server.route('/post', methods=['POST'])
def get_Image_Video():

  pixel = 256
  print("Ana fel function")
  image = request.files['Image']
  video = request.files['Video']
  model = request.form.get('Model') # Fetch the radio button value
  dataset = request.form.get('Dataset')
  print(dataset)

  image.save("/content/imagetest.jpeg")
  video.save("/content/videotest.mp4")
  source_image_path = '/content/imagetest.jpeg'
  driving_video_path = '/content/videotest.mp4'
  output_video_path = '/content/imagetest_videotest.mp4'




  if(model == 'Thin'):
    source_image = imageio.imread(source_image_path)
    reader = imageio.get_reader(driving_video_path)
    source_image = resize(source_image, (pixel, pixel))[..., :3]
    if(dataset == 'fashion'):
      fps = reader.get_meta_data()['fps']
    driving_video = []
    try:
        for im in reader:
            driving_video.append(im)
    except RuntimeError:
        pass
    reader.close()
    driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

    from thin.demo import make_animation
    from skimage import img_as_ubyte
    if predict_mode=='relative' and find_best_frame:
        from thin.demo import find_best_frame as _find
        i = _find(source_image, driving_video, device.type=='cpu')
        print ("Best frame: " + str(i))
        driving_forward = driving_video[i:]
        driving_backward = driving_video[:(i+1)][::-1]
        if(dataset == 'fashion'):
          fps = reader.get_meta_data()['fps']
          predictions_forward = make_animation(source_image, driving_forward, inpaintingFashion, kp_detectorFashion, dense_motion_networkFashion, avd_networkFashion, device = device, mode = predict_mode)
          predictions_backward = make_animation(source_image, driving_backward, inpaintingFashion, kp_detectorFashion, dense_motion_networkFashion, avd_networkFashion, device = device, mode = predict_mode)
        else:
          predictions_forward = make_animation(source_image, driving_forward, inpaintingGif, kp_detectorGif, dense_motion_networkGif, avd_networkGif, device = device, mode = predict_mode)
          predictions_backward = make_animation(source_image, driving_backward, inpaintingGif, kp_detectorGif, dense_motion_networkGif, avd_networkGif, device = device, mode = predict_mode)
        predictions = predictions_backward[::-1] + predictions_forward[1:]
    else:
      if(dataset == 'fashion'):
        predictions = make_animation(source_image, driving_video, inpaintingFashion, kp_detectorFashion, dense_motion_networkFashion, avd_networkFashion, device = device, mode = predict_mode)
      else:
        predictions = make_animation(source_image, driving_video, inpaintingGif, kp_detectorGif, dense_motion_networkGif, avd_networkGif, device = device, mode = predict_mode)

    #save resulting video
    if(dataset == 'fashion'):
      imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps = fps)
    else:
      imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions])

    
  elif model == 'Latent':
    os.chdir("/content/LIA")
    print("here")
    if(dataset == "fashion"):
      subprocess.call(["python", "run_demo.py", "--model", "fashion", "--source_path", source_image_path, "--driving_path", driving_video_path])
      print("done")
      output_video_path = "/content/LIA/res/fashion/imagetest_videotest.mp4"
    elif(dataset == "gif"):
      subprocess.call(["python", "run_demo.py", "--model", "gif", "--source_path", source_image_path, "--driving_path", driving_video_path])
      print("done")
      output_video_path = "/content/LIA/res/gif/imagetest_videotest.mp4"
  
  if os.path.exists(output_video_path):
      print("Success")
      return send_file(output_video_path)
  else:
      return 'Error'


server.run()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://fb53-34-143-146-88.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Ana fel function
fashion


100%|██████████| 425/425 [00:30<00:00, 13.97it/s]
INFO:werkzeug:127.0.0.1 - - [08/May/2023 14:19:23] "POST /post HTTP/1.1" 200 -


Success
Ana fel function
fashion
here


...

done
Success
Ana fel function
gif


100%|██████████| 17/17 [00:01<00:00, 13.14it/s]
INFO:werkzeug:127.0.0.1 - - [08/May/2023 14:21:12] "POST /post HTTP/1.1" 200 -


Success
Ana fel function
gif
here


INFO:werkzeug:127.0.0.1 - - [08/May/2023 14:21:33] "POST /post HTTP/1.1" 200 -


done
Success
